## Assignment 3
### Ex. 3
***Consider the simple linear regression model:***
$$
y_i = \beta_0 + \beta_1x_i + \epsilon_i, \quad i = 1, ..., N
$$
***where $\beta_0$ and $\beta_1$ are the unknown parameters. Assume that the $\epsilon_i$'s are iid $t$-distributed data with unknown degrees of freedom $\nu$.***

***(a) Write the loglikelihood function for the MLE estimation of the three unknowns parameters***.

Our model can be written as
$$
y \mid x \sim \tau(\beta_0 + \beta_1x, \nu)
$$
where $\tau(\mu, \eta)$ is a (noncentral) Student's $t$-distribution centered in $\mu$. 
The loglikelihood $l$ of our model is
$$
l\left((\beta_0, \beta_1, \nu \mid y, x \right) = \log \prod_{i=1}^N f(y_i - \beta_0 - \beta_1x_i \mid \nu) = \sum_{i=1}^N \log f(y_i - \beta_0 - \beta_1x_i \mid \nu)
$$
where $f(\cdot \mid \nu)$ is the density function of a standard Student's $t$-distibution with $\nu$ degrees of freedom, given by
$$
f(x \mid \nu) =  \left\{\sqrt{\nu}B\left(\frac{1}{2},
\frac{\nu}{2}\right)\right\}^{-1} \left(1 + \frac{x^2}{\nu}\right)^{-\frac{\nu + 1}{2}} 
$$
so the log density takes the form

$$
l\left((\beta_0, \beta_1, \nu \mid y, x \right) = \sum_{i = 1}^N  \left\{ - \log \left(\sqrt{v} B\left(\frac{1}{2}, \frac{\nu}{2}\right) \right) - \frac{1}{2}(\nu + 1)\log\left( 1 + \frac{(y_i - \beta_0 - \beta_1x_i)^2}{\nu} \right) \right\}
$$

In [1]:
function logdt(x, v) 
    return -log(sqrt(v) * beta(0.5, 0.5*v)) - 0.5(v + 1)*log(1 + x^2/v)
end

logdt (generic function with 1 method)

In [2]:
function loglikelihood(β0, β1, v, y, x)
    return sum(logdt.(y - β0 - β1*x, v))
end

loglikelihood (generic function with 1 method)

In [3]:
using DataFrames

In [4]:
tdata = readtable("data/tdata.tsv");

In [35]:
using JuMP
using NLopt
m = Model(solver=NLoptSolver(algorithm=:LD_LBFGS))
fobj(β0, β1) = -loglikelihood(β0, β1, 0.05, tdata[:y], tdata[:x])
end
@variable(m, β0)
@variable(m, β1)
# @variable(m, v >= 0.01)
JuMP.register(m, :fobj, 2, fobj, autodiff=true)
@NLobjective(m, Min, fobj(β0, β1))

In [36]:
solve(m)
getobjectivevalue(m)

192.72742240694976

In [7]:
result = optimize(fobj, [2., 3., 10], BFGS())

LoadError: [91mUndefVarError: BFGS not defined[39m

In [8]:
fobj([10, -1., 0.00000000001])

LoadError: [91mMethodError: no method matching fobj(::Array{Float64,1})[0m
Closest candidates are:
  fobj(::Any, [91m::Any[39m, [91m::Any[39m) at In[5]:5[39m

In [9]:
Pkg.add("NLopt")

INFO: INFO: METADATA is out-of-date — you may not have the latest version of NLopt
INFO: Use `Pkg.update()` to get the latest versions of your packages
Package NLopt is already installed
